Plan of attack:
- create a pipeline in Ilastik that can classify regions as stroma or non-stroma
- Produce a binary mask where non-stroma regions are '1' and stroma regions are '0'
- Logical AND this mask with all three channels on the fovs such that we are only left with non-zero intensities
    in non-stroma regions
- find spots using bandpass filter and blob-detector
- construct codebook according to the type of assay used (will need to read off file-names etc)
- produce the count x cell matrix, where there is only 'one' cell which is non-stroma region
- estimate total area of non-stroma region in pixels (could just sum elements of binary mask matrix)
- produce density of target genes, i.e. number of spots/pixel area, for each channel

Some notes:
- binary mask will be the same across all channels
- Check in with Ashley about whether the stroma and non-stroma classification is valid
- could either use the binary mask to force pixel intensities to be zero, or just use it as a label image for the 
    codebook gene assignment

In [2]:
from InSituToolkit.imaging_database import write_experiment
import imaging_db.database.db_operations as db_ops
import imaging_db.filestorage.s3_storage as s3_storage
import imaging_db.filestorage.local_storage as local_storage
import imaging_db.utils.db_utils as db_utils
import os, csv, pickle

%gui qt5
import numpy as np
from skimage import io
from starfish import Experiment, display, Codebook, ExpressionMatrix, FieldOfView, BinaryMaskCollection, LabelImage, ImageStack
from starfish.image import Filter
from starfish.spots import FindSpots, DecodeSpots, AssignTargets
from starfish.types import Axes, Coordinates, Features, FunctionSource, TraceBuildingStrategies

from InSituToolkit.analysis import save_stack

db_credentials = '/Users/andrew.cote/Documents/db_credentials.json'
root_path = '/Users/andrew.cote/Documents/In-Situ_Transcriptomics/LungInSitu/'

def produceMaskFromTif(path: str, img_stack: ImageStack):
    """
    Converts a segmentation TIF file into a binary mask which can be used for assigning spots to cells
    """
    label_image = io.imread(path)

    # we must also get the physical coords from the original to produce the binary mask
    yc = img_stack.xarray.yc.values
    xc = img_stack.xarray.xc.values
    physical_ticks = {Coordinates.Y: yc, Coordinates.X:xc}

    y = img_stack.xarray.y.values
    x = img_stack.xarray.x.values
    pixel_coords = {Axes.Y: y, Axes.X: x}

    label_im = LabelImage.from_label_array_and_ticks(label_image, \
                                                pixel_ticks=pixel_coords, \
                                                physical_ticks=physical_ticks, \
                                                log = img_stack.log)

    mask = BinaryMaskCollection.from_label_image(label_im)
    return mask, label_image

In [3]:
list_of_datasets = pickle.load(open('list_of_experiments.obj', 'rb'))
dict_of_datasets = pickle.load(open('dict_of_experiments.obj', 'rb'))
CODEBOOK = pickle.load(open('codebook.obj', 'rb'))

In [4]:
""" ALREADY RAN ONCE
for dataset in list_of_datasets:
    save_path, exp_name, assayNo = dict_of_datasets[dataset]

    exp = Experiment.from_json(save_path + 'experiment.json')
    exp_name_safe = exp_name.replace('/', '-')

    # get all fovs from the experiment
    fovs = [k for k in exp.keys()]

    # To import images into Ilastik, we want to take the max projection of z-Planes across all color channels, 
    # since the stroma tissue flouresces brightly and we want to paint these regions in Ilastik. 
    for fov in fovs:
        img_stack = next(exp[fov].get_images(FieldOfView.PRIMARY_IMAGES))
        img_stack_sel = img_stack.sel({Axes.CH: 0})
        img_stack_reduced = img_stack_sel.reduce({Axes.ZPLANE}, func='max')
        save_stack(img_stack_reduced, root_path + 'ilastik/reduced_fovs/' + exp_name_safe + fov + '.tif')
"""

" ALREADY RAN ONCE\nfor dataset in list_of_datasets:\n    save_path, exp_name, assayNo = dict_of_datasets[dataset]\n\n    exp = Experiment.from_json(save_path + 'experiment.json')\n    exp_name_safe = exp_name.replace('/', '-')\n\n    # get all fovs from the experiment\n    fovs = [k for k in exp.keys()]\n\n    # To import images into Ilastik, we want to take the max projection of z-Planes across all color channels, \n    # since the stroma tissue flouresces brightly and we want to paint these regions in Ilastik. \n    for fov in fovs:\n        img_stack = next(exp[fov].get_images(FieldOfView.PRIMARY_IMAGES))\n        img_stack_sel = img_stack.sel({Axes.CH: 0})\n        img_stack_reduced = img_stack_sel.reduce({Axes.ZPLANE}, func='max')\n        save_stack(img_stack_reduced, root_path + 'ilastik/reduced_fovs/' + exp_name_safe + fov + '.tif')\n"

In [86]:
'''
At this point we have exported all the reduced / projected images and then classified them using Ilastik
'''
SpotFinder = FindSpots.BlobDetector(min_sigma=1,
                                    max_sigma=10,
                                    num_sigma=10,
                                    threshold=0.01,
                                    measurement_type='mean'
                                   )

glp = Filter.GaussianLowPass(sigma=1)
ghp = Filter.GaussianHighPass(sigma=3)

# TODO: iterate over all datasets: 
dataset = list_of_datasets[0]

save_path, exp_name, assayNo = dict_of_datasets[dataset]
codebook = Codebook.from_code_array(CODEBOOK[int(assayNo)])
exp = Experiment.from_json(save_path + 'experiment.json')

gene_counts_across_fovs = []

for fov in exp.keys():
    # 1 - already completed
    # 2: Filter images and project Zplanes
    img_stack = next(exp[fov].get_images(FieldOfView.PRIMARY_IMAGES))
    img_stack_f1 = ghp.run(img_stack,  in_place=False)
    img_stack_f2 = glp.run(img_stack_f1,  in_place=False)
    img_proj_z = img_stack_f2.reduce({Axes.ZPLANE}, func='max')

    # 2: Find Spots
    spots = SpotFinder.run(img_proj_z)
    decoder = DecodeSpots.SimpleLookupDecoder(codebook=codebook)
    decoded_intensities = decoder.run(spots=spots)

    # 3:
    path = root_path + 'ilastik/classified_fovs/' + exp_name + fov + '.tif'
    mask, label_image = produceMaskFromTif(path, img_stack)

    # 4:
    al = AssignTargets.Label()
    labeled = al.run(mask, decoded_intensities)
    cg = labeled.to_expression_matrix()

    # 5:
    STROMA_ID = 2
    NON_STROMA_ID = 1

    xdiff = np.mean(np.diff(img_stack.xarray.xc.values))
    ydiff = np.mean(np.diff(img_stack.xarray.yc.values))

    area = xdiff * ydiff

    pixel_area = np.sum(label_image == NON_STROMA_ID)
    physical_area = pixel_area * area

    # NOTE: highly likely that 0 == cell_id for non-stroma tissue, yet to be determined however

    gene_counts = {}
    gene_counts['area'] = physical_area

    for target in codebook.indexes['target']:
        gene_counts[target] = cg.loc[0, target].values

    gene_counts_across_fovs.append(gene_counts)
    print(fov + ' completed')

100%|██████████| 21/21 [00:06<00:00,  3.35it/s]


fov_000 completed


100%|██████████| 21/21 [00:09<00:00,  2.30it/s]


fov_001 completed


100%|██████████| 21/21 [00:07<00:00,  2.92it/s]


...

fov_009 completed


  0%|          | 0/21 [00:00<?, ?it/s]

fov_010 completed


100%|██████████| 21/21 [00:07<00:00,  2.85it/s]

fov_011 completed


In [90]:
total_area = 0
gene_density = {'IGFBP3': 0}

for item in gene_counts_across_fovs:
    total_area += item['area']
    gene_density['IGFBP3'] = gene_density['IGFBP3'] + item['IGFBP3']
    

dens = gene_density['IGFBP3']/total_area

In [91]:
dens

0.001059533199127473